# QUICHE

In [ ]:
import os
import pandas as pd
import numpy as np
import quiche as qu
import scanpy as sc
import anndata
import dill
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline

sc.set_figure_params(dpi = 400, dpi_save = 400, fontsize = 14)

## Setup data

In [ ]:
base_dir = "."
cell_tab_path = os.path.join(base_dir, "data", "cell_table_size_normalized.csv")
metadata_path = os.path.join(base_dir, "data", "metadata.csv")

cell_tab = pd.read_csv(cell_tab_path)
metadata = pd.read_csv(metadata_path)
data = pd.merge(cell_tab, metadata, on="fov", how="left")

# A little reformatting
data.index = data['fov'].astype(str) + '_cell' + data['label'].astype(str)
data['cell_meta_cluster'] = data['cell_meta_cluster'].astype('category')

In [ ]:
# Define a list of markers
markers = ['CD11c', 'CD14', 'CD163', 'CD20','CD21', 'CD31', 'CD3e',
           'CD4', 'CD45', 'CD45RO', 'CD56', 'CD68', 'CD69', 'CD86', 
           'CD8a', 'CXCR5', 'Calprotectin', 'Caspase1', 'FOXP3', 
           'Galectin9', 'Glut1', 'GranzymeB', 'HH3', 'HIV1p24',
           'HLA1classABC', 'HLADR', 'ICOS', 'IDO', 'IFNg', 'Ki67',
           'Lag3', 'MastCellTryptase', 'NLRP3', 'PD1', 'PDL1', 
           'SMA', 'TCF1TCF7', 'TIGIT', 'TIM3', 'Vimentin']

# Create an AnnData object with the selected functional marker columns from the DataFrame 'data'
adata = anndata.AnnData(data.loc[:, markers].astype('float'))

# Assign observations metadata to the AnnData object
adata.obs = data.loc[:, ['fov','centroid-0', 'centroid-1', 'label', 'cell_meta_cluster', 'sample_id', 'status']]

# Assign spatial coordinates to the AnnData object
adata.obsm['spatial'] = data.loc[:, ['centroid-0', 'centroid-1']]

# Ensure observation names are strings
adata.obs_names = adata.obs_names.astype('str')

# Normalize
adata.raw = adata
adata.X = qu.pp.standardize(adata.X)

# Filter
sketch_size = 50000
adata  = qu.pp.filter_fovs(adata, 'sample_id', sketch_size)

## Perform differential spatial enrichment analysis with QUICHE

In [ ]:
## initialize class
quiche_op = qu.tl.QUICHE(adata = adata, labels_key = 'cell_meta_cluster', spatial_key = 'spatial', fov_key = 'fov', patient_key = 'sample_id', segmentation_label_key = 'label')
## step 1: compute spatial niches 
quiche_op.compute_spatial_niches(radius = 200, n_neighbors = 30, min_cell_threshold = 3)
## step 2: perform distribution-focused downsampling
quiche_op.subsample(sketch_size = sketch_size, sketch_key = 'sample_id', n_jobs = 8)
## step 3: test for differential spatial enrichment across conditions
quiche_op.differential_enrichment(design = '~status', model_contrasts = 'statushiv_pos-statushiv_neg', k_sim = 100)
## step 4: annotate niche neighborhoods
quiche_op.annotate_niches(nlargest = 3, annotation_scheme = 'neighborhood', annotation_key = 'quiche_niche_neighborhood')

In [ ]:
mdata = quiche_op.mdata
all_out = mdata['quiche'].var

In [ ]:
# Save output to file
with open('../quiche/quiche_hivpos_hivneg_sketch50000.pkl', 'wb') as file:
    dill.dump(quiche_op, file)

## Inspect data

In [ ]:
with open('../quiche/quiche_hivpos_hivneg_sketch50000.pkl', 'rb') as file:
    quiche_op = dill.load(file)

### Access significant niche neighborhoods

In [ ]:
niche_scores = qu.tl.filter_niches(quiche_op,
                                   thresholds = {'logFC': {'median' : [-1, 1]}, 'SpatialFDR': {'median' : 0.05}},
                                   min_niche_count = 5,
                                   annotation_key = 'quiche_niche_neighborhood')

niche_scores.sort_values(by = 'logFC').head()

In [ ]:
niche_metadata = qu.tl.compute_niche_metadata(quiche_op,
                                            niches = list(niche_scores.index),
                                            annotation_key = 'quiche_niche_neighborhood',
                                            patient_key = 'sample_id',
                                            condition_key = 'status',
                                            niche_threshold = 0,
                                            condition_type  = 'binary',
                                            metrics = ['logFC', 'SpatialFDR', 'PValue'])

niche_metadata.head()

### Plot significant niche neighborhoods of interest

In [ ]:
qu.pl.beeswarm_proportion(quiche_op,
                 niche_metadata = niche_metadata,
                niches = niche_scores.index,
                xlim = [-10,10],
                xlim_proportion = [-1, 1],
                logfc_key = 'logFC',
                pvalue_key = 'SpatialFDR',
                annotation_key = 'quiche_niche_neighborhood',
                condition_key = 'status',
                figsize = (6, 12),
                fontsize = 10,
                colors_dict = {'hiv_neg': '#377eb8', 'hiv_pos': '#e41a1c'},
                save_directory = '../quiche/plots',
                filename_save = 'beeswarm_proportion')

### Plot niche network diagrams

In [ ]:
colors_dict = {'myeloid':'#4DCCBD', 'lymphoid':'#279AF1', 'other_immune':'#B07AA1', 'other':'#FF8484', 'structural':'#F9DC5C'}

lineage_dict = {'APC':'myeloid', 'B':'lymphoid', 'CD11c': 'myeloid', 'CD11c_CD14': 'myeloid', 'CD11c_CD68': 'myeloid', 'CD14': 'myeloid',
                'CD14_CD68_CD163': 'myeloid', 'CD3_other': 'lymphoid', 'CD4T': 'lymphoid', 'CD68': 'myeloid', 'CD8T': 'lymphoid',
                'Endothelial':'structural', 'FDC': 'other_immune', 'Foxp3': 'lymphoid', 'Immune_other':'other_immune', 'Mast':'other_immune',
                'NK':'lymphoid', 'Neutrophils':'other_immune', 'Other':'other', 'SMA':'structural', 'Tfh':'lymphoid', 'Unassigned':'other'}

cell_ordering = ['APC', 'CD11c', 'CD11c_CD14', 'CD11c_CD68', 'CD14',
                 'CD14_CD68_CD163', 'CD68', 'B', 'CD8T', 'CD4T', 'Tfh',
                 'Foxp3', 'CD3_other', 'NK', 'FDC', 'Mast', 'Neutrophils',
                 'Immune_other', 'SMA', 'Endothelial', 'Other', 'Unassigned']

##niche network for patients that did not relapse, ie logFC < 0
niche_metadata_neg = niche_metadata[(niche_metadata['mean_logFC'] < 0) & (niche_metadata['n_patients_niche'] > 1)]
G1 = qu.tl.compute_niche_network(niche_df = niche_metadata_neg,
                          colors_dict = colors_dict,
                          lineage_dict = lineage_dict,
                          annotation_key = 'quiche_niche_neighborhood')

qu.pl.plot_niche_network_donut(G = G1,
                               figsize=(6, 6),
                               node_order = cell_ordering,
                               centrality_measure = 'eigenvector',
                               colors_dict = colors_dict,
                               lineage_dict=lineage_dict, 
                               donut_radius_inner = 1.15,
                               donut_radius_outer = 1.25,
                               edge_cmap = 'bone_r',
                               edge_label = 'Samples',
                               save_directory = '../quiche/plots',
                               filename_save = 'hivneg_donut')

##niche network for patients that relapsed, ie logFC > 0
niche_metadata_pos = niche_metadata[(niche_metadata['mean_logFC'] > 0) & (niche_metadata['n_patients_niche'] > 1)]
G2 = qu.tl.compute_niche_network(niche_df = niche_metadata_pos,
                          colors_dict = colors_dict,
                          lineage_dict = lineage_dict,
                          annotation_key = 'quiche_niche_neighborhood')

qu.pl.plot_niche_network_donut(G = G2,
                               figsize=(6, 6),
                               node_order = cell_ordering,
                               centrality_measure = 'eigenvector',
                               colors_dict = colors_dict,
                               lineage_dict=lineage_dict, 
                               donut_radius_inner = 1.15,
                               donut_radius_outer = 1.25,
                               edge_cmap = 'bone_r',
                               edge_label = 'Samples',
                               save_directory = '../quiche/plots',
                               filename_save = 'hivpos_donut')

### Map niche neighborhoods back to FOV for visualization

In [ ]:
mdata = quiche_op.mdata
all_out = mdata['quiche'].var

In [ ]:
annotation_key = "quiche_niche_neighborhood"
quiche_op.mdata['spatial_nhood'][quiche_op.mdata['spatial_nhood'].obs.loc[:, annotation_key] == niche]

#### Plot niche neighborhoods

In [ ]:
niche = 'APC__CD8T'
seg_dir = '../data/segmentation_masks'

qu.pl.plot_niches(quiche_op, 
                  niche = niche,
                  fovs = ['sample2_fov10'],
                  segmentation_directory = seg_dir,
                  save_directory = os.path.join('../quiche/plots', 'quiche_niche_neighborhood'),
                  fov_key = "fov",
                  segmentation_label_key = "label",
                  labels_key = 'cell_meta_cluster',
                  annotation_key = "quiche_niche_neighborhood",
                  seg_suffix = ".tiff",
                  colors_dict = {'APC': '#66cdaa', 'CD8T': '#ff5666'},
                  figsize = (4, 4),
                  dpi = 300)

#### Plot niche scores (e.g. logFC, -log10(SpatialFDR))

In [ ]:
qu.pl.plot_niche_scores(quiche_op, 
                        niche = niche,
                        fovs = ['sample10_fov10'],
                        segmentation_directory = seg_dir,
                        save_directory = os.path.join('figures', 'logFC'),
                        fov_key = "fov",
                        segmentation_label_key  = "label",
                        labels_key='cell_meta_cluster',
                        annotation_key = "quiche_niche_neighborhood",
                        metric = "logFC",
                        seg_suffix = ".tiff",
                        vmin  = None,
                        vmax = None,
                        cmap = "vlag",
                        figsize = (6, 6),
                        dpi = 300)

qu.pl.plot_niche_scores(quiche_op, 
                        niche = niche,
                        fovs = ['sample10_fov10'],
                        segmentation_directory = seg_dir,
                        save_directory = os.path.join('figures', '-log10(SpatialFDR)'),
                        fov_key = "fov",
                        segmentation_label_key  = "label",
                        labels_key='cell_meta_cluster',
                        annotation_key = "quiche_niche_neighborhood",
                        metric = "-log10(SpatialFDR)",
                        seg_suffix = ".tiff",
                        vmin  = None,
                        vmax = None,
                        cmap = "Reds",
                        figsize = (6, 6),
                        dpi = 300)

### plot differential expression within niches as compared to the entire cohort

In [ ]:
functional_markers = ["Caspase1","CD45RO","CD69","CD86","CXCR5","Galectin9",
                      "Glut1","GranzymeB","HLADR","ICOS","IDO","IFNg","Ki67","Lag3",
                      "NLRP3","PD1","PDL1","TCF1TCF7","TIGIT","TIM3","Vimentin"]
quiche_op.compute_functional_expression(niches=niche_scores.index,
                                        annotation_key = 'quiche_niche_neighborhood',
                                        min_cell_threshold = 3,
                                        foldchange_key = 'logFC',
                                        markers = functional_markers,
                                        n_jobs = 8)

In [ ]:
qu.pl.plot_differential_expression(quiche_op,
                            niches = np.unique(niche_metadata_neg.quiche_niche_neighborhood), 
                            annotation_key = 'quiche_niche_neighborhood',
                            labels_key = 'cell_meta_cluster',
                            markers = functional_markers,
                            figsize = (10, 10),
                            cmap = 'vlag',
                            save_directory = '../quiche/plots',
                            filename_save = 'hivneg_matrixplot')

qu.pl.plot_differential_expression(quiche_op,
                            niches = np.unique(niche_metadata_pos.quiche_niche_neighborhood), 
                            annotation_key = 'quiche_niche_neighborhood',
                            labels_key = 'cell_meta_cluster',
                            markers = functional_markers,
                            figsize = (10, 10),
                            cmap = 'vlag',
                            vmin = -1,
                            vmax = 1,
                            vcenter = 0,
                            dendrogram = True,
                            save_directory = '../quiche/plots',
                            filename_save = 'hivpos_matrixplot')